In [169]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.types import StringType, StructType, IntegerType, DateType, StructField
from pyspark.sql.functions import lower, explode, regexp_replace, split, col

In [171]:
spark = SparkSession.builder \
    .appName("Some movies..") \
    .getOrCreate()

In [173]:
path = "C:/Users/VO7/Downloads/movies.csv"

In [175]:
df = spark.read.csv(path, header = True)

In [177]:
df.show(20, 50)

+-----------------------------------+---------+---------------------+----------------+-------------+-----------------+---------------+----+
|                               Film|    Genre|          Lead Studio|Audience score %|Profitability|Rotten Tomatoes %|Worldwide Gross|Year|
+-----------------------------------+---------+---------------------+----------------+-------------+-----------------+---------------+----+
|         Zack and Miri Make a Porno|  Romance|The Weinstein Company|              70|  1.747541667|               64|        $41.94 |2008|
|                    Youth in Revolt|   Comedy|The Weinstein Company|              52|         1.09|               68|        $19.62 |2010|
| You Will Meet a Tall Dark Stranger|   Comedy|          Independent|              35|  1.211818182|               43|        $26.66 |2010|
|                       When in Rome|   Comedy|               Disney|              44|            0|               15|        $43.04 |2010|
|              What 

In [179]:
words = df.select(explode(split(regexp_replace(lower(col("Film")), r"[^a-zA-Z0-9\s]", ""), r"\s+")).alias("word"))

In [181]:
filter_words = set(["the", "and", "a", "of", "to", "in", "is", "it", "you", "that", "for", "on", "with", "as", "this", "by"])

In [183]:
filtered_words_df = words.filter(~col("word").isin(filter_words)) 

In [185]:
word_counts = filtered_words_df.groupBy("word").count()

In [187]:
most_common_word = word_counts.orderBy(col("count").desc()).first()

In [191]:
if most_common_word:
    print(f"Самое частое слово: '{most_common_word['word']}', которое встречается {most_common_word['count']} раз).")
else:
    print("Не найдено ни одного слова после фильтрации.")


Самое частое слово: 'love', которое встречается 4 раз).


In [193]:
spark.stop()